In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r'/content/processed_data.csv')

In [37]:
class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None
        
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        
        return user_items
        
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())
            
        return item_users
        
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
            
        return all_items
        
    def construct_cooccurence_matrix(self, user_songs, all_songs):
        user_songs_users = []        
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)
        
        for i in range(0,len(all_songs)):

            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())
            
            for j in range(0,len(user_songs)):       
                    
                users_j = user_songs_users[j]
                    
                users_intersection = users_i.intersection(users_j)
                
                if len(users_intersection) != 0:
                    users_union = users_i.union(users_j)
                    
                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0
                    
        
        return cooccurence_matrix

    
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))
        
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()
 
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)
    
        columns = ['user_id', 'song', 'score', 'rank']
        df = pd.DataFrame(columns=columns)
         
        rank = 1 
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1
        
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
 
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    def recommend(self, user):
        user_songs = self.get_user_items(user)    
            
        
        all_songs = self.get_all_items_train_data()
        
         
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
                
        return list(df_recommendations.song)

In [38]:
total_play_count = sum(df.freq)
play_count = df[['release', 'freq']].groupby('release').sum().\
             sort_values(by='freq',ascending=False).head(10000)
song_subset = list(play_count.index[:10000])
user_subset = list(df.loc[df.release.isin(song_subset), 'user_id'].unique())
user_song_list_count_sub = df[df.release.isin(song_subset)]

In [39]:
is_model = item_similarity_recommender_py()
is_model.create(user_song_list_count_sub, 'user_id', 'release')

In [40]:
user_id = list(user_song_list_count_sub.user_id)[7]
is_model.recommend(user_id)

Non zero values in cooccurence_matrix :815


['Undo',
 'Revelry',
 'Secrets',
 'Fireflies',
 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)',
 'Marry Me',
 'OMG',
 'Tive Sim',
 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)',
 'Invalid']